# MCP (Model Context Protocol) Demo

This notebook demonstrates how to use MCP (Model Context Protocol) with the .NET Agent Framework.

## What is MCP?

MCP (Model Context Protocol) is a standardized protocol for connecting AI models to external tools and data sources. It allows AI agents to:
- Call external tools/functions
- Access resources (files, databases, APIs)
- Maintain context across interactions

## Demo Overview

In this demo, we'll:
1. View configured MCP servers
2. List available MCP tools
3. Call MCP tools directly
4. Use the Demo API endpoints (which can be exposed via MCP)
5. Simulate MCP-style interactions with local data

In [ ]:
import requests
import json

# Configuration
BASE_URL = "http://localhost:8000"
AUTH_TOKEN = "Bearer demo-token-12345"

headers = {
    "Content-Type": "application/json",
    "Authorization": AUTH_TOKEN
}

def pretty_print(data):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=2))

## 1. Check Demo API Health

First, let's verify the Demo API is running and see available endpoints.

In [ ]:
# Check Demo API health
response = requests.get(f"{BASE_URL}/api/demo/health")
print("Demo API Health:")
pretty_print(response.json())

## 2. View Configured MCP Servers

Let's see what MCP servers are configured in our system.

In [ ]:
# Get configured MCP servers
response = requests.get(f"{BASE_URL}/api/mcp/servers", headers=headers)
print("Configured MCP Servers:")
pretty_print(response.json())

## 3. Demo API Endpoints (REST)

The Demo API provides REST endpoints that simulate a product/order management system. These endpoints can also be exposed via MCP.

### 3.1 Get Products (GET endpoint)

In [ ]:
# Get all products
response = requests.get(f"{BASE_URL}/api/demo/products", headers=headers)
print("All Products:")
pretty_print(response.json())

In [ ]:
# Filter products by category
response = requests.get(
    f"{BASE_URL}/api/demo/products",
    params={"category": "Electronics"},
    headers=headers
)
print("Electronics Products:")
pretty_print(response.json())

In [ ]:
# Filter products by price range
response = requests.get(
    f"{BASE_URL}/api/demo/products",
    params={"minPrice": 100, "maxPrice": 500},
    headers=headers
)
print("Products between $100 and $500:")
pretty_print(response.json())

### 3.2 Get Single Product

In [ ]:
# Get a specific product by ID
response = requests.get(f"{BASE_URL}/api/demo/products/1", headers=headers)
print("Product ID 1:")
pretty_print(response.json())

### 3.3 Create Order (POST endpoint with Authorization)

In [ ]:
# Create a new order (requires Authorization header)
order_request = {
    "customerId": "CUST003",
    "productId": 2,
    "quantity": 5,
    "notes": "Rush delivery requested"
}

response = requests.post(
    f"{BASE_URL}/api/demo/orders",
    json=order_request,
    headers=headers
)
print("Created Order:")
pretty_print(response.json())

In [ ]:
# Try creating order without authorization (should fail)
response = requests.post(
    f"{BASE_URL}/api/demo/orders",
    json=order_request,
    headers={"Content-Type": "application/json"}  # No Authorization header
)
print(f"Status Code: {response.status_code}")
print("Response (should be Unauthorized):")
pretty_print(response.json())

### 3.4 Get Customer Orders

In [ ]:
# Get orders for a customer
response = requests.get(
    f"{BASE_URL}/api/demo/orders/customer/CUST001",
    headers=headers
)
print("Orders for CUST001:")
pretty_print(response.json())

## 4. MCP-Style Simulation

The MCP controller provides a simulation endpoint that demonstrates how MCP tools would interact with local data, similar to how an MCP server would expose these operations.

In [ ]:
# Simulate MCP call: get_products
mcp_request = {
    "operation": "get_products",
    "arguments": {
        "category": "Electronics"
    }
}

response = requests.post(
    f"{BASE_URL}/api/mcp/demo/simulate",
    json=mcp_request,
    headers=headers
)
print("MCP Simulation - get_products:")
pretty_print(response.json())

In [ ]:
# Simulate MCP call: get_product
mcp_request = {
    "operation": "get_product",
    "arguments": {
        "id": 1
    }
}

response = requests.post(
    f"{BASE_URL}/api/mcp/demo/simulate",
    json=mcp_request,
    headers=headers
)
print("MCP Simulation - get_product:")
pretty_print(response.json())

In [ ]:
# Simulate MCP call: create_order
mcp_request = {
    "operation": "create_order",
    "arguments": {
        "customerId": "MCP_CUSTOMER",
        "productId": 3,
        "quantity": 2
    }
}

response = requests.post(
    f"{BASE_URL}/api/mcp/demo/simulate",
    json=mcp_request,
    headers=headers
)
print("MCP Simulation - create_order:")
pretty_print(response.json())

In [ ]:
# Simulate MCP call: get_order
mcp_request = {
    "operation": "get_order",
    "arguments": {
        "id": 1
    }
}

response = requests.post(
    f"{BASE_URL}/api/mcp/demo/simulate",
    json=mcp_request,
    headers=headers
)
print("MCP Simulation - get_order:")
pretty_print(response.json())

## 5. Using the MCP Server (stdio transport)

The MCP Server project (`McpServer/`) exposes the Demo API endpoints as proper MCP tools. This can be started separately and connected via stdio transport.

### Starting the MCP Server

```bash
# From the Backend/dotnet directory:
cd McpServer
dotnet run
```

### MCP Server Tools

The MCP server exposes these tools:
- `get_products` - List products with optional filters
- `get_product` - Get a specific product by ID
- `create_order` - Create a new order
- `get_order` - Get order details by ID
- `get_customer_orders` - Get orders for a customer
- `health_check` - Check Demo API health

### Configuration

MCP servers can be configured in `config.yml`:

```yaml
mcp:
  enabled: true
  servers:
    - name: "demo-api-server"
      transport: "stdio"
      command: "dotnet"
      arguments:
        - "run"
        - "--project"
        - "McpServer/DotNetAgentFramework.McpServer.csproj"
      environmentVariables:
        DEMO_API_BASE_URL: "http://localhost:8000"
        DEMO_API_AUTH_TOKEN: "Bearer demo-token-12345"
```

## 6. Integration with AI Agents

The MCP tools can be used by AI agents to interact with external systems. Here's how to use the chat endpoint with MCP-enabled agents.

In [ ]:
# Chat with an agent about products (agent can use MCP tools internally)
chat_request = {
    "message": "What products do you have in the Electronics category?",
    "agents": ["azure_openai_agent"],
    "format": "detailed"
}

response = requests.post(
    f"{BASE_URL}/chat",
    json=chat_request,
    headers=headers
)
print("Chat Response:")
pretty_print(response.json())

## Summary

This demo showed:

1. **Demo REST API** - Sample endpoints with GET and POST operations requiring authorization
2. **MCP Configuration** - How to configure MCP servers via config.yml or environment variables
3. **MCP Client Service** - Internal service for connecting to MCP servers
4. **MCP Controller** - REST API for managing MCP connections and calling tools
5. **MCP Server** - Separate project that exposes REST endpoints as MCP tools

### Key Files

- `Controllers/DemoController.cs` - Sample REST endpoints
- `Controllers/McpController.cs` - MCP management API
- `Services/McpClientService.cs` - MCP client implementation
- `Configuration/McpConfig.cs` - MCP configuration models
- `McpServer/` - Standalone MCP server project
- `config.yml` - Configuration including MCP server definitions